In [1]:
import pandas as pd
import numpy as np
import collections
import time
import math

In [2]:
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=',',
                           usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties'])
item_metadata[:3]

,item_id,n_clicks,views,clicks_views_ratio,sum_of_properties
0,5001,3.0,99,0.030303,48.0
1,5002,6.0,68,0.088235,55.0
2,5003,6.0,90,0.066667,73.0


In [66]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 3000000)
#                    nrows=10000
                   )
train = train[train["action_type"] == "clickout item"]
print(train.shape)
train[:3]

(973484, 13)


,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,3000012,32f395007948b,1541136919,J3TS5897X8ZA,2,clickout item,101399,MY,"Kuala Lumpur, Malaysia",mobile,NaN,101399|101404|2395937|101373|116290|101403|128...,70|78|127|48|67|103|53|65|94|52|59|32|58|86|38...
19,3000018,5b4388d8108b5,1541136920,I3FI14XOJKM6,3,clickout item,363046,JP,"Seoul, South Korea",desktop,NaN,4841682|363046|2712746|475151|3220960|1237720|...,69|73|77|73|79|58|82|67|67|72|80|70|65|75|62|5...
21,3000020,bbcfdb1763063,1541136920,21BSN2NNRYN2,13,clickout item,7789474,IN,"Darjeeling, India",desktop,NaN,1004901|9731784|9398258|2012685|7789474|271509...,29|10|27|16|25|30|16|20|7|16|24|15|22|19|18|24...


In [67]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [ ]:
train = explode_impressions_and_prices(train)
train[:3]

In [ ]:
# # DODANIE CITY I COUNTRY
# city, country = (train['city']
#                     .values[0]
#                     .replace(" ", "")
#                     .split(','))
# print(city)
# print(country)

# cities = []
# countries = []
# l = []
# for index in range(len(train['city'].values)):
#     l = (train['city']
#                     .values[index]
#                     .replace(" ", "")
#                     .split(','))
#     cities.append(l[0])
#     countries.append(l[1])
# #     if len(l) > 2 :
# #         print(l[0])
# #         print(l[1])
# #         print(l[2])
# #         print(len(l))

# train = train.drop(['city'], axis=1)
# train['city'] = cities
# train['country'] = countries
# train[:3]

In [ ]:
train = train.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
train[:3]

In [ ]:
train = train.rename({'index':'item_id'})
train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','sum_of_properties']]
train['reference'] = train['reference'].astype(int)
train[:3]

In [ ]:
train['clicked'] = np.where(train['item_id'] == train['reference'], 1, 0)
train[:3]

# NAUKA

In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train[['n_clicks','views','clicks_views_ratio']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
rus = RandomUnderSampler(random_state=0)
X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
# X_train = normalize(X_train, norm='l2')
X_test = sc.transform(X_test)
# X_test = normalize(X_test, norm='l2')

In [ ]:
logreg = LogisticRegression(n_jobs=-1, class_weight={0.0:0.85, 1.0:0.15}) #class_weight={0.0:0.2, 1.0:0.8}
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# y_pred_prob = logreg.predict_proba(X_test)

In [ ]:
print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

# ZBIÓR TESTOWY

In [53]:
test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\test.csv',
                       sep=','
#                     skiprows=range(1, 11000000)
#                    nrows=100000
                   )
# test = test[test["action_type"] == "clickout item" & test['reference'].isnull()]
mask = test["reference"].isnull() & (test["action_type"] == "clickout item")
test = test[mask]
# print(test.shape)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
9,U9MAV55LZMOR,1541453209,0dd5544c71728,1,clickout item,NaN,US,"Atlantic City, USA",mobile,NaN,366186|56563|3741160|63568|3478738|3137008|678...,69|68|46|39|69|47|48|52|74|70|44|34|57|69|37|7...
15,VXIWZ3NF0WHF,1541453212,be1b5623b2e12,3,clickout item,NaN,DK,"Copenhagen, Denmark",tablet,NaN,4412788|53876|3147220|929429|148127|53835|1753...,122|144|195|100|92|117|72|107|268|166|171|657|...
16,2VZWKE7NWP8C,1541453213,7daf68934bfef,1,clickout item,NaN,US,"Weston, USA",mobile,NaN,57189|404846|164156|111309|148435|77409|103228...,115|166|115|97|111|87|200|101|114|160|98|97|10...


In [54]:
test = explode_impressions_and_prices(test)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016


In [55]:
index_df = test[["user_id","session_id","timestamp","step","impressions"]]
index_df[:10]

,user_id,session_id,timestamp,step,impressions
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717


In [56]:
test = test.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,item_id,n_clicks,views,clicks_views_ratio,sum_of_properties
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,3811810.0,238.0,1946.0,0.122302,48.0
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,496141.0,258.0,1951.0,0.132240,76.0
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,674016.0,93.0,2184.0,0.042582,46.0


In [57]:
test = test.rename({'index':'item_id'})
test = test[["item_id", "reference",'n_clicks','views','clicks_views_ratio']]
test = test.drop(columns=['reference'], axis=1)
test = test.dropna()
test[:3]

,item_id,n_clicks,views,clicks_views_ratio
0,3811810.0,238.0,1946.0,0.122302
1,496141.0,258.0,1951.0,0.132240
2,674016.0,93.0,2184.0,0.042582


In [58]:
X_test = test[['n_clicks','views','clicks_views_ratio']]
X_test = sc.transform(X_test)

In [59]:
y_pred = logreg.predict_proba(X_test)

In [60]:
y_pred

array([[0.62767753, 0.37232247],
       [0.58897707, 0.41102293],
       [0.85694745, 0.14305255],
       ...,
       [0.92220733, 0.07779267],
       [0.85193557, 0.14806443],
       [0.78910495, 0.21089505]])

In [61]:
result = pd.concat([index_df, pd.DataFrame(y_pred)], axis=1, sort=False)
result = result.drop([0], axis=1)
result[:10]

,user_id,session_id,timestamp,step,impressions,1
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810,0.372322
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141,0.411023
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016,0.143053
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426,0.336696
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848,0.102409
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372,0.134266
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816,0.145615
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336,0.114429
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061,0.144369
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717,0.099623


In [62]:
result = result.assign(impressions=lambda x: x["impressions"].apply(str))
result = result.sort_values(["user_id", "session_id", "timestamp", "step", 1],
                     ascending=[True, True, True, True, False])
result[:10]

,user_id,session_id,timestamp,step,impressions,1
2958366,000I430EXZC0,ddabdf53a4d38,1541501642,1,103600,0.237194
2958356,000I430EXZC0,ddabdf53a4d38,1541501642,1,2111996,0.223616
2958351,000I430EXZC0,ddabdf53a4d38,1541501642,1,103658,0.198166
2958346,000I430EXZC0,ddabdf53a4d38,1541501642,1,3247218,0.196730
2958352,000I430EXZC0,ddabdf53a4d38,1541501642,1,103670,0.153085
2958369,000I430EXZC0,ddabdf53a4d38,1541501642,1,3539818,0.152849
2958361,000I430EXZC0,ddabdf53a4d38,1541501642,1,824196,0.150551
2958347,000I430EXZC0,ddabdf53a4d38,1541501642,1,8805412,0.140023
2958354,000I430EXZC0,ddabdf53a4d38,1541501642,1,1148728,0.134448
2958368,000I430EXZC0,ddabdf53a4d38,1541501642,1,693506,0.128672


In [63]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [64]:
result = group_concat(result, ["user_id","session_id","timestamp","step"], "impressions")
result = result.rename(columns={"impressions":"item_recommendations"})
result[:10]

,user_id,session_id,timestamp,step,item_recommendations
0,000I430EXZC0,ddabdf53a4d38,1541501642,1,103600 2111996 103658 3247218 103670 3539818 8...
1,000O8CPC6T0W,846b712ae8e1f,1541480658,1,5961668 51012 335561 13876 52993 923037 6506 1...
2,000REY2YYH1D,4a313f2eb8954,1541465923,1,355861 65392 62474 65491 57541 63809 56901 580...
3,0010PD3FH7SV,26b9e7e18cc87,1541535269,2,689911 6345002 4410388 1709889 10368292 798866...
4,0012IHLNTQWC,a035e580f0e64,1541505846,39,2502093 1307328 1100198 2041695 1515725 265934...
5,0015CRBC6OH9,8bcbd478ac2cf,1541519175,3,48460 2179674 3375754 50434 12086 50270 600246...
6,0018EAWF0OIK,10ce4a98a476f,1541513087,2,8230426 998597 7851978 7892712 4072668 1000092...
7,001EBIZM062K,0e036884adc53,1541502068,1,42262 42276 42291 9866100 42176 42134 42186 48...
8,001G88F8NP9P,2a2e9eb9c967c,1541511909,9,513106 3165837 7202328 6078754 7125170 7915180...
9,001JFV0AYE6Z,73ad1e08b4430,1541481561,1,67196 596076 9043150 971641 58414 70440 59822 ...


In [65]:
result.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_ML.csv", index=False,)